In [ ]:
def makeeye(sourcepath,savename,date,stt):
    import cv2, dlib
    import numpy as np
    from imutils import face_utils
    from keras.models import load_model
    import moviepy.editor as mpe
    import subprocess
    import pandas as pd
    
    IMG_SIZE = (34, 26)
    
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('model/shape_predictor_68_face_landmarks.dat')
    
    model = load_model('model/videoopencv.h5')
    #model.summary()
    
    def crop_eye(img, eye_points):
        x1, y1 = np.amin(eye_points, axis=0)
        x2, y2 = np.amax(eye_points, axis=0)
        cx, cy = (x1 + x2) / 2, (y1 + y2) / 2
        w = (x2 - x1) * 1.2
        h = w * IMG_SIZE[1] / IMG_SIZE[0]
        margin_x, margin_y = w / 2, h / 2
        min_x, min_y = int(cx - margin_x), int(cy - margin_y)
        max_x, max_y = int(cx + margin_x), int(cy + margin_y)
        eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)
        eye_img = gray[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]
        return eye_img, eye_rect
    
    def get_mspec(movie):
        length = int(movie.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(movie.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(movie.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = movie.get(cv2.CAP_PROP_FPS)
        return length, width, height, fps
    
    def convert_video(video_input, video_output):
        cmds = ['ffmpeg', '-i', video_input, '-r', '24', video_output]
        p = subprocess.Popen(cmds, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = p.communicate()
        return stdout, stderr
    
    # main
    
    # 눈깜빡임 수를 담을 변수
    frame_counter = 1
    frame_index = []
    blink_counter = 0
    blink_check = True
    blinking = 0
    
    #영상파일 이름
    video_input = sourcepath
    video_output = "result/"+savename+date+'/'+savename+date+'24fps.mp4'
    resultpath = "result/"+savename+date+'/'+savename+date+'24fps_result.mp4'
    
    convert_video(video_input, video_output)
    
    cap = cv2.VideoCapture(video_output)
    cap_length, cap_width, cap_height, cap_fps_n = get_mspec(cap)
    print(cap_length, cap_width, cap_height, cap_fps_n)
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(resultpath, fourcc, cap_fps_n, (cap_width//2, cap_height//2))
    
    while cap.isOpened():
        
        ret, img_ori = cap.read()
        
        #frame이 정상적으로 받아지는지 체크
        if not ret:
            break
    
        # frame resizing
        img_ori = cv2.resize(img_ori, dsize=(0, 0), fx=0.5, fy=0.5)
        img = img_ori.copy()
        
        if frame_counter==1:
            img_with_box=img
    
        # 분석할 프래임 선택 작업 (둘 중에 1개 분석하지 않음)
        if frame_counter%2!=0:
        
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = detector(gray)
    
            for face in faces:
                shapes = predictor(gray, face)
                shapes = face_utils.shape_to_np(shapes)
    
                eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
                eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])
    
                eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
                eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)
                eye_img_r = cv2.flip(eye_img_r, flipCode=1)
    
    #            cv2.imshow('lefteye', eye_img_l)
    #            cv2.imshow('righteye', eye_img_r)
    
                eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
                eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
    
                pred_l = model.predict(eye_input_l)
                pred_r = model.predict(eye_input_r)
    
                # visualize
                CNST = 0.01
                state_l = 'O %.1f' if pred_l > CNST else '- %.1f'
                state_r = 'O %.1f' if pred_r > CNST else '- %.1f'
    
                if pred_l < CNST and pred_r < CNST:
                    blinking = 1
                    if blink_check:
                        blink_check=False
                        blink_counter += 1
                else:
                    blinking = 0
                    blink_check=True
    
                state_l = state_l % pred_l
                state_r = state_r % pred_r
    
                cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(255,255,0), thickness=1)
                cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(255,255,0), thickness=1)
    
                cv2.putText(img, state_l, tuple(eye_rect_l[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255,255,255), 1)
                cv2.putText(img, state_r, tuple(eye_rect_r[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255,255,255), 1)
    
                img_with_box = img.copy()
                # for문 끝
            #>>> 선택된 프래임 분석 작업 끝
        # if문 끝
            
        #프래임 저장 및 출력
        frame_index.append([frame_counter, pred_l, pred_r])
        video.write(img_with_box)
        #cv2.imshow('result', img_with_box) # 분석과정 화면에 출력
        
        if cv2.waitKey(1) == ord('q'): # 분석 중 'q' 키를 누르면 분석을 멈춤.
            break
        
        frame_counter += 1
    # while문 끝
    
    # 모두 닫기
    cap.release()
    video.release()
    cv2.destroyAllWindows()
    
    arr = np.array(frame_index)
    from matplotlib import font_manager, rc
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
    %matplotlib inline
    import matplotlib.pyplot as plt
    resultlist = []
    for i in range(0,len(arr)//24):
        left = 0
        right = 0
        for j in range(i*24,(i+1)*24):
            left = left+arr[j,1]
            right = right+arr[j,2]
        resultlist.append([j,left/24,right/24])
    resultarr = np.array(resultlist)
    
    

    
    
    
    
    
    plt.figure(figsize=(30, 8))
    plt.title('눈 깜박임 차트',fontdict = {'fontsize' : 50})
    plt.plot(resultarr[:,1])
    plt.plot(resultarr[:,2])
    
    stt = pd.DataFrame(stt,columns=['word','start','end'])
    df = pd.DataFrame(range(1,len(resultarr)+1),columns=['time'])
    df['str'] = ''
    for i in range(0,len(stt)):
        df['str'][df['time']==stt.loc[i,'start'].astype(int)] = df['str'][df['time']==stt.loc[i,'start'].astype(int)] + stt.loc[i,'word'] 
    s = np.pi/4
    n = -1
    for i in range(0,len(df)):
        s = s+np.pi/4
        n = n*-1
        yp = resultarr[i,int(1.5+n*0.5)] -0.03+np.sin(s)*0.03
        if (yp<0):
            yp = 0.01
        if (yp>1):
            yp = 0.095
        plt.text(i-0.5,yp ,df['str'][i],fontdict={'fontsize':12})
    
    plt.title('눈 떨림 분석 차트',fontdict = {'fontsize' : 50})
    plt.legend(['Left Eye','Right Eye'],loc='best')
    plt.xlabel('값이 0에 가까울수록 눈을 깜박임', fontdict={'fontsize':20})
    plt.savefig("result/"+savename+date+'/'+savename+date+'eyegraph.png')
    print("eye graph saved in "+"result/"+savename+date+'/'+savename+date+'eyegraph.png')
    
#    result_cap = cv2.VideoCapture(resultpath)
#    result_cap_length, result_cap_width, result_cap_height, result_cap_fps_n = get_mspec(result_cap)
#    print(result_cap_length, result_cap_width, result_cap_height, result_cap_fps_n)
#    result_cap.release()
#    
#    # sync를 위한 fps 계산
#    duration = cap_length/cap_fps_n
#    final_fps_n=result_cap_length/duration
#    
#    # 중간 비디오클립의 ftp 맞추기
#    videoclip = mpe.VideoFileClip(resultpath)
#    orig_clip = mpe.VideoFileClip(video_output)
#    audioclip = orig_clip.audio
#    final_clip = videoclip.set_audio(audioclip)
#    final_clip.write_videofile("result/"+savename+date+'/'+savename+date+'eyevideo.mp4')
#    print("eye video saved in "+"result/"+savename+date+'/'+savename+date+'eyevideo.mp4')
#    
#    import os
#    try:
#        os.remove('result/'+name+date+'/'+name+'44100.wav')
#        os.remove('result/'+name+date+'/'+name+date+'24fps.mp4')
#        os.remove('result/'+name+date+'/'+name+date+'24fps_result.mp4')
#        os.remove('result/'+name+date+'/'+name+date+'_final.mp4')
#    except:
#        print('remove error')